# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [38]:
from imblearn.metrics import geometric_mean_score



In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [11]:
# Create our features

X = df.drop(columns=["loan_status"])
X = pd.get_dummies(X) 

# Create our target

y = df['loan_status']


In [12]:
# Check the balance of our target values
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [13]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    )

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [14]:
# Create the StandardScaler instance
data_scaler = StandardScaler()

In [15]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = data_scaler.fit(X_train)

In [16]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [21]:
from imblearn.ensemble import BalancedRandomForestClassifier

In [22]:
# Resample the training data with the BalancedRandomForestClassifier

rf_model = BalancedRandomForestClassifier(n_estimators=500, random_state=78)
rf_model.fit(X_train_scaled, y_train.ravel())

BalancedRandomForestClassifier(n_estimators=500, random_state=78)

In [23]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score


In [25]:
predictions = rf_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, predictions)

0.7673111031870259

In [26]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[   64,    37],
       [ 1694, 15410]])

In [28]:
# Print the imbalanced classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions, target_names=target_names))

              precision    recall  f1-score   support

Class Purple       0.04      0.63      0.07       101
Class Yellow       1.00      0.90      0.95     17104

    accuracy                           0.90     17205
   macro avg       0.52      0.77      0.51     17205
weighted avg       0.99      0.90      0.94     17205



In [54]:
# List the features sorted in descending order by feature importance
feature_importance = rf_model.feature_importances_
indices = np.argsort(feature_importance)[::-1]
for i in range(X.shape[1]):
    print(f"{X.columns.values[indices[i]]}: ({feature_importance[indices[i]]})")

total_rec_prncp: (0.06589012765240791)
last_pymnt_amnt: (0.06345960536756735)
total_pymnt_inv: (0.06255293872526813)
total_rec_int: (0.05551963273263345)
total_pymnt: (0.05525035457144288)
int_rate: (0.03136484056458762)
installment: (0.01829008879689813)
mths_since_recent_inq: (0.01772029178774874)
issue_d_Jan-2019: (0.017643180392030437)
dti: (0.017083302518324354)
out_prncp_inv: (0.016742057097896577)
out_prncp: (0.016599986409617892)
max_bal_bc: (0.015740011480467976)
tot_hi_cred_lim: (0.014905024204032046)
revol_bal: (0.014715960843834449)
annual_inc: (0.014587071411433627)
avg_cur_bal: (0.014455664387447987)
mths_since_rcnt_il: (0.014186728674112305)
total_rev_hi_lim: (0.01400694212642123)
total_bc_limit: (0.013859747120657516)
mo_sin_old_il_acct: (0.01380014643366348)
issue_d_Mar-2019: (0.013538926349260088)
il_util: (0.013522420420286643)
bc_open_to_buy: (0.013352750398010831)
bc_util: (0.01332652700459957)
mo_sin_old_rev_tl_op: (0.013141883688722395)
total_bal_il: (0.012405308

In [43]:
print('The geometric mean is {}'.format(geometric_mean_score(
    y_test, predictions)))

The geometric mean is 0.9339629740854085


### Easy Ensemble Classifier

In [32]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
eec_model = EasyEnsembleClassifier(n_estimators=500, random_state=78)
eec_model.fit(X_train_scaled, y_train.ravel())

EasyEnsembleClassifier(n_estimators=500, random_state=78)

In [50]:
# Calculated the balanced accuracy score
predictions1 = eec_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, predictions1)

0.9340571715029314

In [51]:
# Display the confusion matrix
confusion_matrix(y_test, predictions1)

array([[   93,     8],
       [  901, 16203]])

In [52]:
# Print the imbalanced classification report
print(classification_report(y_test, predictions1, target_names=target_names))

              precision    recall  f1-score   support

Class Purple       0.09      0.92      0.17       101
Class Yellow       1.00      0.95      0.97     17104

    accuracy                           0.95     17205
   macro avg       0.55      0.93      0.57     17205
weighted avg       0.99      0.95      0.97     17205



In [53]:
print('The geometric mean is {}'.format(geometric_mean_score(
    y_test, predictions1)))

The geometric mean is 0.9339629740854085


### Final Questions

1. Which model had the best balanced accuracy score?

    Easy Ensemble 

2. Which model had the best recall score?

    Easy Ensemble 

3. Which model had the best geometric mean score?

    They appear to be the same. 

4. What are the top three features?

total_rec_prncp: (0.06589012765240791)
last_pymnt_amnt: (0.06345960536756735)
total_pymnt_inv: 0.06255293872526813)